In [1]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import pickle
import overview
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [15]:
#Cut it the way you want
importlib.reload(AtTools)
importlib.reload(overview)
anubis_file = "proAnubis_240627_1720.raw"
atlas_file = "dataAtlas.root"


#chunks, times, fReader = overview.get_chunks(anubis_file, start="2024-06-27 16:55:20", end = "2024-06-27 16:55:20")

analyser = AtTools.AtlasAnalyser()
#analyser.fReader = fReader
analyser.saveTDC5Data(dir+anubis_file, dir+"anubis_data.pkl", 1719503720, 1719506560)
#analyser.getTDC5Data(dir+"abubis_data.pkl", 4,  2, amount_of_events=200)
#print("Done")
#data = analyser.getAtlasData(dir+atlas_file)
#analyser.atlas_data = analyser.atlas_data[85_000:100_000]
#print("Anubis", (round(analyser.anubis_data[0].timeStamp), round(analyser.anubis_data[-1].timeStamp)))
#print("ATLAS",(analyser.atlas_data.iloc[0]["TimeStamp"], analyser.atlas_data.iloc[-1]["TimeStamp"]))


Initial time 2024-06-27 16:20:18.087426


Skipping Events proAnubis_240627_1720.raw: 100%|██████████| 2102/2102 [05:49<00:00,  6.01Events/s] 


Skip time 2024-06-27 16:55:20.029632


Processing Chunks proAnubis_240627_1720.raw: 1seconds [00:00, 125.09seconds/s]


Ending time: 2024-06-27 16:55:20.048321


In [18]:
start = 1719503720
end = 1719506560
TDC5Reads  = []
current_timestamp = start
with tqdm(total=round(end - start), desc=f"Reading", unit="seconds") as pbar: 
        while current_timestamp < end:
            if not fReader.readBlock():
                raise EOFError("You have reached the end of the file")
            
            if(fReader.hasEvents()):
                tdc5Reads = fReader.getTDCFiveEvents()
                if not tdc5Reads:
                    continue
            else:
                continue
            TDC5Reads.append(tdc5Reads)
            current_timestamp = datetime.timestamp(tdc5Reads[0][0])
            pbar.update(round(current_timestamp - pbar.n))

storage = "try.pkl"
with open(storage, "wb") as f:
    pickle.dump(TDC5Reads, f)
    print("Saved")
#print(TDC5Reads)

Reading: 1719505116seconds [08:36, 2477.96seconds/s]       

In [33]:
times = analyser.atlas_data["TimeStamp"]
print(times) 
#1719504220
#1719503530

hist, bins = np.histogram(times, bins=100)
plt.plot(bins[:-1], hist)
plt.show()

118       1719498206
29        1719498206
340       1719498206
190       1719498206
153       1719498206
             ...    
144140    1719506684
145102    1719506684
145264    1719506684
144324    1719506684
144329    1719506684
Name: TimeStamp, Length: 145275, dtype: uint32


In [6]:
# set the times
print("Anubis", (round(analyser.anubis_data[0].timeStamp), round(analyser.anubis_data[-1].timeStamp)))
print("ATLAS",(analyser.atlas_data.iloc[7000]["TimeStamp"], analyser.atlas_data.iloc[-1]["TimeStamp"]))

Anubis (1719501618, 1719501980)
ATLAS (np.uint32(1719503671), np.uint32(1719506684))


In [9]:
#Save the Anubis data
with open("anubis_data1.pkl", "wb") as f:
    pickle.dump(analyser.anubis_data, f)

In [79]:
#Calculate the BCR offset
importlib.reload(AtTools)
anubis = AtTools.BCRHistogram(analyser.anubis_data, atlas=False, plot = False)
atlas = AtTools.BCRHistogram(analyser.atlas_data, atlas=True, plot = False)

window_size = 100
anubis = np.convolve(np.append(anubis, anubis[:window_size]), np.ones(window_size)/window_size, mode='valid')
atlas = np.convolve(np.append(atlas, atlas[:window_size]), np.ones(window_size)/window_size, mode='valid')
error = 100
errors = []
for offset in range(-200,200):
    errors.append(np.sum((atlas - np.roll(anubis, offset))**2))
    if np.sum((atlas - np.roll(anubis, offset))**2) < error:
        error = np.sum((atlas - np.roll(anubis, offset))**2)
        best_offset = offset

print("Best offset", best_offset)
plt.plot(atlas, label="Atlas")
#plt.plot(anubis, label="Original Anubis")
plt.plot(np.roll(anubis, best_offset), label="Offset Anubis")
plt.xlim(0, 3565)
plt.ylabel("Counts (normalized)")
plt.xlabel("BX")
plt.title("Anubis and Atlas BCR Histograms aligned - smoothed")
plt.legend()
plt.show()

plt.plot(list(range(-200,200)), errors, label="Errors")
plt.xlabel("Offset")
plt.ylabel("Sum of Square Errors")
plt.title("Sum of Square Errors vs Offset")
plt.vlines(best_offset,0, max(errors), color = "red", label=f"Best Offset: {best_offset}")
plt.ylim(0,max(errors))
plt.legend(loc = "lower left") #position

plt.show()
    

Best offset 114


In [3]:
importlib.reload(AtTools)
analyser.pairBCRwithEvents()

Matching: 100%|██████████| 14500/14500 [21:22<00:00, 11.31Events/s]  


[[RunNumber                                                 479103
  EventNumber                                                    0
  LumiBlock                                                    553
  TimeStamp                                             1719505383
  TimeStampNS                                            636189985
  BCID                                                        1626
  extL1ID                                               2164269924
  triggerType                                                  140
  mu_pt          [76.32294, 33.715176, 14.507515, 10.183615, 9....
  mu_eta         [-1.9750708, -0.9686851, 0.451308, 0.45357943,...
  mu_phi         [-1.6176077, -2.8990316, -0.28081444, -0.31152...
  mu_E           [280.32965, 50.80989, 16.010548, 11.249753, 15...
  mu_charge                                    [-1, 1, 1, 1, 1, 1]
  Name: 83671, dtype: object,
  []],
 [RunNumber                       479103
  EventNumber                          0
  LumiBloc

In [5]:
#difference
best_offset = 114
eta = []
phi = []
times = []
matches = analyser.matches
for i in range(len(matches)):
    if matches[i][1]:
        for trigger in matches[i][1]:
            #if abs(round(trigger.bcId-matches[i][0]["BCID"])) > 1000:
            #    continue
            eta.extend(matches[i][0]["mu_eta"])
            phi.extend(matches[i][0]["mu_phi"])       
            for _ in range(len(matches[i][0]["mu_eta"])):
                times.append(round(trigger.bcId+best_offset-matches[i][0]["BCID"]))


In [86]:
#3D plot
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(eta, phi, times)
ax.set_xlabel('eta')
plt.ylim(-3.2,3.2)
plt.xlim(-3.2,3.2)
ax.set_ylabel('phi')
ax.set_zlabel('time')
plt.show()

In [6]:
#Histograms
plt.hist2d(eta, phi, bins=30, cmap='plasma')
plt.xlim(-np.pi, np.pi)
plt.ylim(-np.pi, np.pi)
plt.xlabel("eta")
plt.ylabel("phi")
# Add a color bar to show the color scale
plt.colorbar()
plt.show()

plt.hist(times, bins=range(min(times), max(times) + 2, 1))
#plt.xlim(-100,100)
plt.xlabel("Time difference (bx)")
plt.ylabel("Frequency")
plt.title("Time difference between BCR and trigger")
plt.show()